In [1]:
import torch   
import torch.nn as nn
import copy
from torch.functional import F

### AbstractTorch
Ceci est un prototype de moteur d'évaluation s'appuyant sur Torch. L'idée est de surcharger nn.Linear avec des méthodes hybrides, gérant un flux abstrait pour l'évaluation et un flux concret. La simultanéité des deux flux permet d'oberver la position du centre du zonotope par rapport à la valeur réelle de la sortie de la fonction. 
Ce moteur est moins précis que Saimple car il ne génère pas de nouveaux symboles mais additionne les approximations dans un symbole poubelle. 

L'implémentation du modèle abstrait comporte une quantité fixe de symboles qui sont gérés comme des épaisseurs de batch. La dernière épaisseur de batch correspond au symbole poubelle, les opération linéaires sont opérées pour cette épaisseur par la valeur absolu de la matrice des poids. 

Pour l'instant sont implémentées les classes conv2D, Linear , et ReLU. 
        #TODO implémenter maxpool2D, ... 
    

L'idée est de tirer profit de la classe nn.Module de Torch en la surchargeant avec des méthodes mixes (flux concret et abstrait). On tire profit de la structure de base
de la méthode forward. Au lieu de considérer un batch, on considère une entrée en dimension 0 avec dans les dimensions habituelles du batch des couches de symbole. Une couche (un épaisseur de batch) représente
un symbole abstrait. La dernière couche correspond au symbole poubelle. 


La couche 0 représente le centre du zonotope
Les couches suivantes représentent les symboles. Elles sont calculées pour les opération linéaires (Linear et Conv2D) par 
$$\textbf{W}(x_\epsilon)+\textbf{b}-(\textbf{W}(0)+\textbf{b})$$
    x[1:]=lin(x_epsilon)-lin(torch.zeros_like(x_epsilon))

La derniere couche (bruit poubelle) est calculée par

$$\textbf{|W|}(x_\epsilon)+\textbf{b}-(\textbf{|W|}(0)+\textbf{b})$$


Pour implémenter le tenseur linéaire représentant la valeur absolue, on duplique la couche lin ou conv et on applique la valeur absolue à la matrice de poids. 



La méthode abstractTensor permet, à partir d'un tenseur d'origine, de générer un tenseur abstrait. 

Cette méthode doit être largement enrichie en classe, avec différentes méthodes telles que 
    add_noise
    mul_noise
    ...


In [2]:
def abstractTensor(tensor: torch.Tensor, alpha : list[float]):
    assert len(alpha)==len(tensor.flatten()), "The length of alpha should be equal to the length of the flatten tensor"
    print(tensor.shape)
    
    flatten_tensor  = tensor.flatten()
    abstract_tensor=[]
    abstract_tensor.append(tensor)
    for i in range(1,len(flatten_tensor)+1):
        abstract_tensor_layer = torch.zeros_like(flatten_tensor)
        abstract_tensor_layer[i-1]=alpha[i-1]
        abstract_tensor_layer = abstract_tensor_layer.reshape(tensor.shape)
        abstract_tensor.append(abstract_tensor_layer)
    
    abstract_tensor.append(torch.zeros_like(tensor))
    abstract_tensor= torch.stack(abstract_tensor)

    
    return abstract_tensor

In [5]:
class AbstractNN(nn.Module):
    
    def __init__(self,num_symbols: int,num_depth=1,device=torch.device("cpu")):

        super(AbstractNN,self).__init__()
        num_symbols = num_symbols +2
        self.num_symbols = num_symbols
        self.num_depth = num_depth
        self.device = device

        



    @staticmethod    
    def abstract_linear(in_features:int,out_features:int,x,x_true,device=torch.device("cpu")):
        x =x.unsqueeze(1).to(device)
        x_true=x_true.to(device)
        lin = nn.Sequential(nn.Flatten(),
                            nn.Linear(in_features=in_features, out_features=out_features,device=device))
        lin_abs = copy.deepcopy(lin).to(device)
        lin_abs[1].weight.data =torch.abs(lin[1].weight.data)
     
        
        x_value = x[0].unsqueeze(1)
        x_epsilon= x[1:-1].unsqueeze(1)
        x_noise =x[-1].unsqueeze(1)
       
        x=lin(x)    
        x_true = lin(x_true)
        x[0]=lin(x_value)
        x[1:-1]=lin(x_epsilon)-lin(torch.zeros_like(x_epsilon))
        x[-1]=lin_abs(x_noise)-lin_abs(torch.zeros_like(x_noise))
        x_min = x[0] - torch.sum(torch.abs(x[1:]),dim=0)
        x_max = x[0] + torch.sum(torch.abs(x[1:]),dim=0)
        print(x_min.type())
        return x,x_min,x_max,x_true
    
    @staticmethod
    def abstract_conv2D(in_channels:int,out_channels:int,kernel_size:int,x,x_true,device=torch.device("cpu")):
        x=x.to(device)
        x_true = x_true.to(device)
        conv = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,device=device)
      
        conv_abs = copy.deepcopy(conv).to(device)
        conv_abs.weight.data = torch.abs(conv.weight.data)
     
       
        x_value = x[0]
        x_epsilon= x[1:-1]
        x_noise = x[-1]
        x=conv(x)
        x[0]=conv(x_value)
        x_true = conv(x_true)
        x[1:-1]=conv(x_epsilon)-conv(torch.zeros_like(x_epsilon).to(device))
        x[-1]=conv_abs(x_noise)-conv_abs(torch.zeros_like(x_noise).to(device))
        x_min = x[0] - torch.sum(torch.abs(x[1:]),dim=0)
        x_max = x[0] + torch.sum(torch.abs(x[1:]),dim=0)
        
        return x,x_min,x_max,x_true
    
    @staticmethod
    def abstract_relu_conv2D(x,x_min,x_max,x_true,num_symbols:int,add_symbol=False,device=torch.device("cpu")):
        num_symbols = len(x)
        sgn_min = torch.sign(x_min)
        sgn_max = torch.sign(x_max)
        sgn = sgn_min+sgn_max
        p = x_max/(torch.abs(x_max)+torch.abs(x_min))
        q = x_max*(1-p)/2
        d = torch.abs(q)
        x_true  = nn.ReLU()(x_true)
        copy_x_for_approx = x
        mask_p = (sgn==0)*1
        mask_1 =(sgn==2)*1 + (sgn==1)*1
        mask_p = mask_p.unsqueeze(0).expand(num_symbols,-1,-1,-1)
        mask_1 = mask_1.unsqueeze(0).expand(num_symbols,-1,-1,-1)
        
        #approximation of the center 0, (p*x+q) or the value itself 
        copy_x_for_approx[0]= mask_p[0]*(p*copy_x_for_approx[0]+q)+mask_1[0]*copy_x_for_approx[0]
        
        #update of the epsilon
        copy_x_for_approx[1:-1]=p*mask_p[1:-1]*copy_x_for_approx[1:-1] + mask_1[1:-1]*copy_x_for_approx[1:-1]

        #update of the noise symbol -> projection 0, |W|*espilon_noise or new noise if new linear approximation
        copy_x_for_approx[-1]=d*mask_p[-1] +copy_x_for_approx[-1]*mask_p[-1] +mask_1[-1]*copy_x_for_approx[-1]

        x=copy_x_for_approx
        
        x_min = x[0] - torch.sum(torch.abs(x[1:]),dim=0)
        x_max = x[0] + torch.sum(torch.abs(x[1:]),dim=0)
        
        if add_symbol:
            """
            new_symbols_indexes =torch.where(x[-1]!=0)


            for index,value in enumerate(new_symbols_indexes[0]):
            
            
                x =torch.cat((x,x[-1].unsqueeze(0)),dim=0)
                
                x[-2]=torch.zeros_like(x[-2])
                
                x[-2][value]=x[-1][value]
            x[-1]=torch.zeros_like(x[-1])
            """
                        
            new_eps =torch.where(x[-1]!=0)[0].to(device)
            index = torch.arange(len(new_eps)).to(device)
            new_eps_batch_shape = x[-1].expand(len(new_eps)+1,-1,-1,-1).shape
            new_eps_batch = torch.zeros(new_eps_batch_shape).to(device)
            new_eps_batch[index,new_eps]=x[-1][new_eps]

            x=x[:-1]

            x = torch.cat((x,new_eps_batch),dim=0)            

        return x,x_min,x_max,x_true
    
    @staticmethod
    def abstract_relu(x,x_min,x_max,x_true, num_symbols:int,add_symbol=False,device=torch.device("cpu")):
        num_symbols = len(x)
        if add_symbol:
            num_symbols = len(x)
        sgn_min = torch.sign(x_min)
        sgn_max = torch.sign(x_max)
        sgn = sgn_min+sgn_max
        p = x_max/(torch.abs(x_max)+torch.abs(x_min))
        q = x_max*(1-p)/2
        d = torch.abs(q)
        x_true  = nn.ReLU()(x_true)
        copy_x_for_approx = x

        #mask for values that will be approximated by the linear approximation
        mask_p = (sgn==0)*1
        #mask for the values for those the output is the same as the input (y=x)
        mask_1 =(sgn==2)*1+ (sgn==1)*1
        #expand the mask to the number of symbols
        mask_p = mask_p.unsqueeze(0).expand(num_symbols,-1)
        mask_1 = mask_1.unsqueeze(0).expand(num_symbols,-1)
        #approximation of the center  
        copy_x_for_approx[0]= mask_p[0]*(p*copy_x_for_approx[0]+q)+mask_1[0]*copy_x_for_approx[0]
        #uptade of the epsilon
        copy_x_for_approx[1:-1]=p*mask_p[1:-1]*copy_x_for_approx[1:-1] + mask_1[1:-1]*copy_x_for_approx[1:-1]
        #update of the noise symbol -> projection 0, |W|*espilon_noise or new noise if new linear approximation
        copy_x_for_approx[-1]=d*mask_p[-1] +copy_x_for_approx[-1]*mask_p[-1] + mask_1[-1]*copy_x_for_approx[-1]
        x=copy_x_for_approx

        x_min = x[0] - torch.sum(torch.abs(x[1:]),dim=0)
        x_max = x[0] + torch.sum(torch.abs(x[1:]),dim=0)
        
        if add_symbol:
            """
            new_symbols_indexes =torch.where(x[-1]!=0)


            for index,value in enumerate(new_symbols_indexes[0]):
            
            
                x =torch.cat((x,x[-1].unsqueeze(0)),dim=0)
                
                x[-2]=torch.zeros_like(x[-2])
                
                x[-2][value]=x[-1][value]
            x[-1]=torch.zeros_like(x[-1])
            
            """
            new_eps =torch.where(x[-1]!=0)[0].to(device)
            
            index = torch.arange(len(new_eps)).to(device)
            new_eps_batch_shape = x[-1].expand(len(new_eps)+1,-1).shape
            new_eps_batch = torch.zeros(new_eps_batch_shape).to(device)
            new_eps_batch[index,new_eps]=x[-1][new_eps]

            x=x[:-1]

            x = torch.cat((x,new_eps_batch),dim=0)

        return x,x_min,x_max,x_true


    def forward(self,x):

        
        x_true = x
        x_true = x_true[0].unsqueeze(0)
        print(f"lenx:{len(x)}")
        x,x_min,x_max,x_true = self.abstract_conv2D(self.num_depth,16,3,x,x_true,device=self.device)
        x,x_min,x_max,x_true = self.abstract_relu_conv2D(x,x_min,x_max,x_true,self.num_symbols,add_symbol=True,device =self.device)
        print(f"lenx:{len(x)}")
       
        x,x_min,x_max,x_true = self.abstract_conv2D(16,16,3,x,x_true,device=self.device)
        x,x_min,x_max,x_true = self.abstract_relu_conv2D(x,x_min,x_max,x_true,self.num_symbols,add_symbol=True,device =self.device)
        print(f"lenx:{len(x)}")
        x,x_min,x_max,x_true = self.abstract_conv2D(16,32,3,x,x_true,device=self.device)
        x,x_min,x_max,x_true = self.abstract_relu_conv2D(x,x_min,x_max,x_true,self.num_symbols,add_symbol=True,device =self.device)
        print(f"lenx:{len(x)}")
        x,x_min,x_max,x_true = self.abstract_conv2D(32,32,3,x,x_true,device=self.device)
        x,x_min,x_max,x_true = self.abstract_relu_conv2D(x,x_min,x_max,x_true,self.num_symbols,add_symbol=True,device =self.device)
        print(f"lenx:{len(x)}")
        x,x_min,x_max,x_true = self.abstract_linear(86528,6272,x,x_true,device=self.device)
        x,x_min,x_max,x_true = self.abstract_relu(x,x_min,x_max,x_true,self.num_symbols,add_symbol=True,device =self.device)
        print(f"lenx:{len(x)}")
        x,x_min,x_max,x_true = self.abstract_linear(6272,6272,x,x_true,device=self.device)
        x,x_min,x_max,x_true = self.abstract_relu(x,x_min,x_max,x_true,self.num_symbols,add_symbol=True,device =self.device)
        print(f"lenx:{len(x)}")
        x,x_min,x_max,x_true = self.abstract_linear(6272,6272,x,x_true,device=self.device)
        x,x_min,x_max,x_true = self.abstract_relu(x,x_min,x_max,x_true,self.num_symbols,add_symbol=True,device =self.device)
        print(f"lenx:{len(x)}")
        x,x_min,x_max,x_true = self.abstract_linear(6272,6272,x,x_true,device=self.device)
        x,x_min,x_max,x_true = self.abstract_relu(x,x_min,x_max,x_true,self.num_symbols,add_symbol=True,device =self.device)
        print(f"lenx:{len(x)}")
        x,x_min,x_max,x_true = self.abstract_linear(6272,512,x,x_true,device=self.device)
        x,x_min,x_max,x_true = self.abstract_relu(x,x_min,x_max,x_true,self.num_symbols,add_symbol=True,device =self.device)
        print(f"lenx:{len(x)}")
        x,x_min,x_max,x_true = self.abstract_linear(512,256,x,x_true,device=self.device)
        x,x_min,x_max,x_true = self.abstract_relu(x,x_min,x_max,x_true,self.num_symbols,add_symbol=True,device =self.device)
        print(f"lenx:{len(x)}")

        x,x_min,x_max,x_true = self.abstract_linear(256,8,x,x_true,device=self.device)
        x,x_min,x_max,x_true = self.abstract_relu(x,x_min,x_max,x_true,self.num_symbols,device =self.device)
        print(f"lenx:{len(x)}")
        return x,x_min,x_max,x_true
        


Ici on évalue un modèle généré aléatoirement, composé de 4 couches de convolution et de 5 couches linéaires, chacunes avec une fonction d'activation ReLU
L'évaluation se fait sur un tenseur [54,54] généré aléatoirement, qui peut correspondre à une image par exemple. 

In [6]:
!PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0
device =torch.device('mps')
tensor = torch.randn(60,60)
print(torch.max(tensor.flatten()  ))
alpha = 0.001*torch.ones_like(tensor.flatten())
abstract_tensor = abstractTensor(tensor,alpha).float()

print(f"abstract_tensor.shape= {abstract_tensor.shape}")
abstract_tensor = abstract_tensor.to(device)
print(f"abstract_tensor.type= {abstract_tensor.type()}")
model = AbstractNN(60*60,device=device)
model = model.to(device)
print(model)

with torch.no_grad():
 
   result,x_min,x_max,x_true=model(abstract_tensor.unsqueeze(1))


print(f"y_min       =  {x_min}")
print(f"y_max       =  {x_max}")
print(f"center Ztp  =  {result[0]}")
print(f"y_true      =  {x_true[:]}")
print(f"y_max-x_min =  {x_max-x_min}")
print(f"Trash symbol=  {result[-1]}")


tensor(3.0586)
torch.Size([60, 60])
abstract_tensor.shape= torch.Size([3602, 60, 60])
abstract_tensor.type= torch.mps.FloatTensor
AbstractNN()
lenx:3602
lenx:3713
lenx:3880
lenx:4284
lenx:4757
torch.mps.FloatTensor
lenx:5027
torch.mps.FloatTensor
lenx:5343
torch.mps.FloatTensor
lenx:5629
torch.mps.FloatTensor
lenx:5843
torch.mps.FloatTensor
lenx:5850
torch.mps.FloatTensor
lenx:5850
torch.mps.FloatTensor
lenx:5850
y_min       =  tensor([0.0335, -0.0000, -0.0000, -0.0000, 0.0158, -0.0000, 0.0226, 0.0274],
       device='mps:0')
y_max       =  tensor([0.0336, 0.0000, 0.0000, 0.0000, 0.0159, 0.0000, 0.0227, 0.0275],
       device='mps:0')
center Ztp  =  tensor([0.0335, -0.0000, -0.0000, -0.0000, 0.0158, -0.0000, 0.0227, 0.0274],
       device='mps:0')
y_true      =  tensor([[0.0335, 0.0000, 0.0000, 0.0000, 0.0158, 0.0000, 0.0227, 0.0274]],
       device='mps:0')
y_max-x_min =  tensor([6.6265e-05, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.9745e-05, 0.0000e+00,
        6.7160e-05, 7.2993e-05], d

In [6]:
print(len(result))
print(len(abstract_tensor))

3655
3602


Ici on évalue un modèle fictif, l'entrée est une image de 26 sur 26 par 3 épaisseur,avec un symbole associé avec chaque dimension d'entrée (3*26*26)
Attention, la dimension de la première couche linéaire doit s'accorder, le code vous renvoiera une erreur (10368 au lieu de 67712)

In [11]:

tensor = torch.randn(3,26,26)
print(torch.max(tensor.flatten()  ))
alpha = 0.000005*torch.ones_like(tensor.flatten())
abstract_tensor = abstractTensor(tensor,alpha)
print(f"abstract_tensor.shape= {abstract_tensor.shape}")
model = AbstractNN(26*26*3,num_depth=3)
model.eval()

with torch.no_grad():
 
   result,x_min,x_max,x_true=model(abstract_tensor)


print(f"y_min       =  {x_min}")
print(f"y_max       =  {x_max}")
print(f"center Ztp  =  {result[0]}")
print(f"y_true      =  {x_true[:]}")
print(f"y_max-x_min =  {x_max-x_min}")
print(f"Trash symbol=  {result[-1]}")

tensor(3.2062)
torch.Size([3, 26, 26])
abstract_tensor.shape= torch.Size([2030, 3, 26, 26])
y_min       =  tensor([0.0279, -0.0000, 0.0438, -0.0000, -0.0000, 0.0425, 0.0369, 0.0619])
y_max       =  tensor([0.0282, 0.0000, 0.0441, 0.0000, 0.0000, 0.0428, 0.0372, 0.0623])
center Ztp  =  tensor([0.0281, -0.0000, 0.0439, -0.0000, -0.0000, 0.0427, 0.0370, 0.0621])
y_true      =  tensor([[0.0281, 0.0000, 0.0439, 0.0000, 0.0000, 0.0427, 0.0370, 0.0621]])
y_max-x_min =  tensor([0.0003, 0.0000, 0.0004, 0.0000, 0.0000, 0.0004, 0.0003, 0.0004])
Trash symbol=  tensor([0.0002, 0.0000, 0.0002, 0.0000, 0.0000, 0.0002, 0.0002, 0.0002])


In [91]:
device =torch.device('mps')
tensor = torch.randn(2,2)


In [109]:
print(torch.max(tensor.flatten()  ))
alpha = 0.5*torch.ones_like(tensor.flatten())
abstract_tensor = abstractTensor(tensor,alpha).float()
result,x_min,x_max,x_true = AbstractNN.abstract_linear(4,5,abstract_tensor,abstract_tensor[0].unsqueeze(0))
print(f"result = {result}")
result,x_min,x_max,x_true = AbstractNN.abstract_relu(result,x_min,x_max,x_true,6,add_symbol=False)
print(f"result = {result}")
print(f"y_true = {x_true}")

tensor(1.2963)
torch.Size([2, 2])
torch.FloatTensor
result = tensor([[-0.5611, -0.6019, -1.3127,  0.4239,  0.5597],
        [-0.2410, -0.2443, -0.2251,  0.1972,  0.2159],
        [ 0.0670, -0.1541, -0.1763, -0.0461,  0.0528],
        [ 0.1253, -0.1288,  0.1929, -0.0029,  0.0682],
        [-0.1605, -0.1294, -0.1256,  0.1114,  0.1824],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], grad_fn=<CopySlices>)
result = tensor([[ 4.5138e-04,  1.1391e-03, -0.0000e+00,  4.2393e-01,  5.5974e-01],
        [-6.6451e-03, -1.0174e-02,  0.0000e+00,  1.9718e-01,  2.1587e-01],
        [ 1.8461e-03, -6.4185e-03,  0.0000e+00, -4.6076e-02,  5.2821e-02],
        [ 3.4554e-03, -5.3646e-03,  0.0000e+00, -2.8602e-03,  6.8199e-02],
        [-4.4260e-03, -5.3901e-03,  0.0000e+00,  1.1136e-01,  1.8239e-01],
        [ 1.5921e-02,  2.6208e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
       grad_fn=<CopySlices>)
y_true = tensor([[0.0000, 0.0000, 0.0000, 0.4239, 0.5597]], grad_fn=<ReluBackward0>)


In [36]:
sie =result.size()
sie[0]

6

In [101]:
value = torch.tensor(test[0])
value

/var/folders/52/k6pzm88n5_xdgm1mjpjcgnc80000gn/T/ipykernel_52911/1469605289.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(test[0])


tensor([0, 1, 3, 4])

In [110]:

new_eps =torch.where(x[-1]!=0)
index = torch.arange(len(new_eps[0]))
new_eps_batch = result[-1].expand(len(new_eps[0])+1,-1).shape
new_eps_batch = torch.zeros(new_eps_batch)
new_eps_batch[index,new_eps[0]]=result[-1][new_eps[0]]

x=x[:-1]

x = torch.cat((x,new_eps_batch),dim=0)

In [111]:
result

tensor([[ 4.5138e-04,  1.1391e-03, -0.0000e+00,  4.2393e-01,  5.5974e-01],
        [-6.6451e-03, -1.0174e-02,  0.0000e+00,  1.9718e-01,  2.1587e-01],
        [ 1.8461e-03, -6.4185e-03,  0.0000e+00, -4.6076e-02,  5.2821e-02],
        [ 3.4554e-03, -5.3646e-03,  0.0000e+00, -2.8602e-03,  6.8199e-02],
        [-4.4260e-03, -5.3901e-03,  0.0000e+00,  1.1136e-01,  1.8239e-01],
        [ 1.5921e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  2.6208e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
       grad_fn=<CatBackward0>)

In [ ]:


for index,value in enumerate(test[0]):
   
   
    result =torch.cat((result,result[-1].unsqueeze(0)),dim=0)
    
    result[-2]=torch.zeros_like(result[-2])
    
    result[-2][value]=result[-1][value]
result[-1]=torch.zeros_like(result[-1])


print(result)


In [5]:
import torch

# Supposons que votre tenseur d'origine soit x
x = torch.tensor([1, 2, 0, 0, 3, 0, 4, 5, 0, 6])

# Créer un masque pour les valeurs non nulles dans x
mask = (x != 0).float()

# Nombre de valeurs non nulles
num_non_zero = mask.sum().int().item()

# Créer un tenseur identité de la bonne taille
identity = torch.eye(num_non_zero)

# Appliquer le masque aux valeurs non nulles
masked_values = x[mask.bool()]

# Créer le nouveau tenseur en multipliant le tenseur identité par les valeurs non nulles
new_tensor = identity * masked_values.unsqueeze(1)

print(new_tensor)


tensor([[1., 0., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 3., 0., 0., 0.],
        [0., 0., 0., 4., 0., 0.],
        [0., 0., 0., 0., 5., 0.],
        [0., 0., 0., 0., 0., 6.]])


In [6]:
import torch

# Supposons que votre tenseur d'origine soit x
x = torch.tensor([1, 2, 0, 0, 3, 0, 4, 5, 0, 6])

# Créer un masque pour les valeurs non nulles dans x
mask = (x != 0)

# Extraire les valeurs non nulles de x
non_zero_values = x[mask]

# Créer un tenseur diagonal avec les valeurs non nulles
diagonal_tensor = torch.diag_embed(non_zero_values)

# Créer un tenseur de zéros de la taille souhaitée
zeros_tensor = torch.zeros_like(diagonal_tensor)

# Ajouter les deux tenseurs pour obtenir le résultat final
result_tensor = diagonal_tensor + zeros_tensor

print(result_tensor)


tensor([[1, 0, 0, 0, 0, 0],
        [0, 2, 0, 0, 0, 0],
        [0, 0, 3, 0, 0, 0],
        [0, 0, 0, 4, 0, 0],
        [0, 0, 0, 0, 5, 0],
        [0, 0, 0, 0, 0, 6]])


In [11]:
test = torch.zeros(4,10)
index = torch.tensor([0,1,2,3])
value = torch.tensor([4,5,7,2])
test[index,value]=1

In [12]:
test

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])